In [1]:
import torch
from torch import nn
from torch import Tensor
import requests
import os
from pathlib import Path
from typing import Callable
from functools import partial


URL = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'

In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
from efficientnet_pytorch import EfficientNet
from torchsummary import summary
from glasses.utils.Tracker import Tracker
from glasses.utils.ModuleTransfer import ModuleTransfer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_src = EfficientNet.from_pretrained('efficientnet-b0')

summary(model_src.to(device), (3,224,224))

Loaded pretrained weights for efficientnet-b0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 226, 226]               0
Conv2dStaticSamePadding-2         [-1, 32, 112, 112]             864
       BatchNorm2d-3         [-1, 32, 112, 112]              64
MemoryEfficientSwish-4         [-1, 32, 112, 112]               0
         ZeroPad2d-5         [-1, 32, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 32, 112, 112]             288
       BatchNorm2d-7         [-1, 32, 112, 112]              64
MemoryEfficientSwish-8         [-1, 32, 112, 112]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13        

In [5]:
model_src._global_params

GlobalParams(width_coefficient=1.0, depth_coefficient=1.0, image_size=224, dropout_rate=0.2, num_classes=1000, batch_norm_momentum=0.99, batch_norm_epsilon=0.001, drop_connect_rate=0.2, depth_divisor=8, min_depth=None, include_top=True)

In [6]:
from glasses.nn.models.classification.efficientnet import EfficientNet, MemoryEfficientSwish

model_dst  = EfficientNet(activation=MemoryEfficientSwish)

summary(model_dst.to(device), (3,224,224))

TypeError: super() takes no keyword arguments

In [ ]:
model_dst

In [ ]:
from glasses.nn.models.classification.efficientnet import EfficientNet
model_dst  = EfficientNet().to(device)

x = torch.ones((1, 3, 224, 224)).to(device)
_ = ModuleTransfer(model_src, model_dst)(x)

y_dst = model_dst.eval()(x)
y_src = model_src.eval()(x)

In [ ]:
y_dst

In [ ]:
y_src

In [ ]:
model_src